In [1]:
import open3d as o3d
import numpy as np
import copy
from utils.pyransac3d import *

In [2]:
def extract_outliers(pcd_points, inliers):
    temp = []
    
    for i in range(len(pcd_points)):
        idx = i
        if not idx in inliers:
            temp.append(pcd_points[idx])
            
    p_outliers = np.zeros((len(temp), 3))
    for i in range(len(temp)):
        p_outliers[i] = temp[i]
    return p_outliers

In [3]:
def get_inliers(pcd_points, inliers):
    p_inliers = []
    for i in range(len(pcd_points)):
        idx = i
        if idx in inliers:
            p_inliers.append(pcd_points[idx])

    return p_inliers

In [10]:
pcd = o3d.io.read_point_cloud("krylon.pcd")

In [4]:
ball = o3d.geometry.TriangleMesh.create_sphere(radius=0.4)
# box = o3d.geometry.TriangleMesh.create_box(width=0.3, height=0.4, depth=0.5)

In [6]:
pcd = ball.sample_points_uniformly(5000)
# pcd = box.sample_points_uniformly(5000)

In [7]:
pcd = pcd.uniform_down_sample(every_k_points=5)

In [6]:
o3d.visualization.draw_geometries([pcd])

In [8]:
points = np.asarray(pcd.points)
points.shape

(1000, 3)

In [9]:
geometry_list = [Cuboid(), Sphere(), Cylinder()]

In [11]:
eq, inliers = geometry_list[0].fit(points, thresh=0.005, maxIteration=2000)

len(inliers)

59

In [12]:
inliers

array([208, 213, 269, 281, 338, 393, 396, 404, 406, 407, 410, 414, 433,
       441, 466, 492, 494, 511, 515, 516, 524, 525, 542, 569, 574, 590,
       592, 593, 606, 654, 655, 665, 668, 687, 719, 720, 731, 733, 734,
       735, 737, 741, 748, 752, 789, 801, 808, 816, 817, 852, 865, 866,
       867, 872, 873, 907, 909, 910, 919])

In [27]:
outliers = extract_outliers(points, inliers)

In [28]:
pcd_outliers = o3d.geometry.PointCloud()

In [29]:
pcd_outliers.points = o3d.utility.Vector3dVector(outliers)

In [30]:
o3d.visualization.draw_geometries([pcd_outliers])

In [31]:
eq_, inliers_ = geometry_list[0].fit(outliers, thresh=0.005, maxIteration=700)

In [16]:
center, radius, inliers = geometry_list[1].fit(points, thresh=0.001, maxIteration=2000)

len(inliers)

197

In [18]:
len(get_inliers(points, inliers))

197

In [21]:
center, axis, radius, height, inliers = geometry_list[2].fit(points, thresh=0.005, maxIteration=1000)

len(inliers)

195

In [34]:
pcd_inliers = o3d.geometry.PointCloud()
p_inliers= []
# for i in range(len(inliers)):
#     p_inliers.append(points[inliers[i]])
# pcd_inliers.points = o3d.utility.Vector3dVector(p_inliers)
# pcd_inliers.paint_uniform_color((0,0,0))

for i in range(len(inliers_)):
    p_inliers.append(np.asarray(pcd_outliers.points)[inliers_[i]])
    pcd_inliers.points = o3d.utility.Vector3dVector(p_inliers)
    pcd_inliers.paint_uniform_color((0,0,0))

In [36]:
# o3d.visualization.draw_geometries([pcd,pcd_inliers])
o3d.visualization.draw_geometries([pcd_outliers,pcd_inliers])